## Название документа

Название НПА – это краткий текст, описывающий основную суть и тему документа.
Название часто начинается с предлогов «о» или «об». В тексте НПА название может встречаться
в кавычках или без кавычек. Название не включает в себя тип НПА.

Для примера название документа – это «О заместителе Министра внутренних дел
Российской Федерации – начальнике Следственного департамента Министерства внутренних дел Российской Федерации».

Поле name должно содержать строку с названием в кавычках или без, в зависимости от того, как оно встретилось в тексте НПА. При сравнении предсказанного и настоящего номера в проверяющей системе не учитывается регистр букв, пробелы и символы перевода строки не различаются.

## Орган, принявший акт

Часто автор документа указывается после слова «Принят», но далеко не всегда, например в указе автор - «Президент Российской Федерации».

В рамках задания необходимо для каждого документа определить орган, его принявший. Наименование органов необходимо извлекать в «нормализованном виде»:

* Ярославская областная дума
* Федеральная служба по надзору в сфере образования и науки
* Правительство Удмуртской Республики
* Департамент охраны окружающей среды и природопользования Ярославской области

В обучающей и тестовой выборке около 200 уникальных органов. В тестовой выборке есть органы, которых нет в обучающей.

Поле authority должно содержать наименование органа в нормализованном виде. При сравнении предсказанного и настоящего органа не учитывается регистр букв, пробелы и символы перевода строки не различаются.

### Оценка качества

Допускается нечеткое совпадение. Метрика качества – средний коэффициент Жаккара.

Коэффициент Жаккара вычисляется следующим образом:

$$ L(s_{extr}, s_{gold}) = \frac{intersect}{offset_{start} + intersect + offset_{end}} $$

где $intersect$ - длина самой длинной общей подстроки у выделенной строки и строки из «золотой метки», $offset_{start}$ - наибольшее количество символов до начала общей подстроки в выделенной строке и в строке из «золотой метки», $offset_{end}$ - наибольшее количество символов от конца общей подстроки до конца строки в выделенной строке и в строке из «золотой метки».

Эта метрика принимает значения от 0 до 1 и штрафует как за выделение более короткой строки чем строка из «золотой метки», так и за выделение более длинной строки.

* указ - президент, глава, губернатор

* федеральный закон - Государственная Дума Федерального собрания Российской Федерации

* приказ - управление, министерство, департамент, федеральная служба, комитет, агентство

* распоряжение - правительство, администрация, губернатор, президент, глава, дума

* закон - Государственный Совет, Законодательное собрание, областная дума, Законодательная Дума, Государственный Совет, Народное Собрание, Эл Курултай, Президент, областное Собрание

* постановление - Правительство Российской Федерации, Конституционный Суд, Кабинет министров, 
    Правительство, Администрация, Губернатор, Глава, областной совет, областное Собрание,
    Законодательное собрание

In [1]:
import re
from eval_module import quality
import json
import numpy as np

In [2]:
train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])

In [24]:
PREPR_EXPR = re.compile(r'[^\wа-я"\s\.,;:№\-\)\(]')

def preprocess_doc(doc):
    doc = '\n' + doc.lower()
    doc = re.sub('_', ' ', doc)
    doc = re.sub(' +', ' ', doc)
    doc = re.sub(r'кои ', 'кой ', doc)
    return PREPR_EXPR.sub('', doc)

In [156]:
NAME_EXPR = re.compile(r'(\nоб?\s[а-я]([^\n]+\n)+\n)')
UKAZ_EXPR = re.compile(r'(\nвопросы\s([^\n]+\n)+\n)')
POST_EXPR = re.compile(r'"о\s[^"]+"')
LAST_EXPR = re.compile(r'(об? ([^\n]+\n)+)\n')
POST2_EXPR = re.compile(r'(о внесении изменени[яй] [^\"]+)\"')


def extract_name(doc, doc_type):
    doc = preprocess_doc(doc)
    match = NAME_EXPR.search(doc)
    if match:
        name = match.group(0).strip()
        if doc.find('правительство пензенской области') != -1:
            match = POST2_EXPR.search(name)
            if match:
                return match.group(0).strip()
        return name
    if doc_type == 'указ':
        match = UKAZ_EXPR.search(doc)
        if match:
            return match.group(0).strip()
    match = POST_EXPR.search(doc)
    if match:
        return match.group()
    match = LAST_EXPR.search(doc)
    if match:
        return match.group(0).strip()
    return ""

In [157]:
def extract_authority(doc, doc_type):
    if doc_type == 'федеральный закон':
        return "Государственная Дума Федерального собрания Российской Федерации"
    return ""

In [158]:
def predict(test, doc_types):
    results = []
    for doc, doc_type in zip(test, doc_types):
        authority = extract_authority(doc, doc_type)
        name = extract_name(doc, doc_type)
        prediction = {"type": "",
                      "date": "",
                      "number": "",
                      "authority": authority,
                      "name": name}
        results.append(prediction)
    return results

In [159]:
predicted = predict(train, true_types)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.9825380501,
 'authority_jaccard': 0.048132364,
 'subtasks_improves': 1}

In [160]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted_labels[ids], test_labels[ids])['name_jaccard']}")

федеральный закон: 0.9959043084
постановление: 0.9824316182
приказ: 0.9947019235
распоряжение: 0.9753646302
закон: 0.9367822698
указ: 0.99669746


In [162]:
from eval_module import preprocess, string_jaccard_metric

doc_type = 'постановление'
ids = np.where(true_types == doc_type)

y_test = [(x['name'], x['type']) for x in test_labels[ids]]
y_pred = [x['name'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    test_, pred_ = preprocess(test[0]), preprocess(pred)
    if string_jaccard_metric([test_], [pred_]) < 0.6:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("===============================================================================================")        
        print("===============================================================================================")